# MySQL and PyMySQL
Using MySQL Server and Python library PyMySQL and Pandas to CRUD database and table on RDBMS.

## PyMySQL
- Create database
- Delete database
- Connect database
- Create table
- Read table
- Update table
- Delete table 

## Pandas
- MySQL table to Pandas dataframe
- Pandas dataframe to MySQL table

First set mysql settings (host, password, and more) in json file.
PyMySQL has a "connect()" function to connect mysql server, and "connect()" has a "cursor()" which travels mysql server.

In [1]:
import json
import pymysql

config = open('MySQL_Config.json')
config = json.load(config)

conn = pymysql.connect(host=config["localhost"], user=config["user"], password=config["password"], database='', port=3306)
cursor = conn.cursor()

'CREATE DATABASE DB_NAME' to create database on mysql server.
"cursor.execute(sql)" to execute query on mysql server and save it with "conn.commit()"

In [5]:
# Create a database
def create_db(name=None):
    if name == None:
        print("Please enter database name")
    else:
        conn = pymysql.connect(host=config["localhost"], user=config["user"], password=config["password"], database='', port=3306)
        cursor = conn.cursor()

        sql = 'CREATE DATABASE ' + name
        
        try:
            cursor.execute(sql)
            print("created database", name)
        except Exception as e:
            print(e)
        conn.commit()
        conn.close()

'Drop DATABASE DB_NAME' to delete database on mysql server.

In [6]:
# Delete a database
def delete_db(name=None):
    if name == None:
        print("Please enter database name")
    else:
        conn = pymysql.connect(host=config["localhost"], user=config["user"], password=config["password"], database='', port=3306)
        cursor = conn.cursor()

        sql = 'DROP DATABASE ' + name
        try:
            cursor.execute(sql)
            print("deleted database", name)
        except Exception as e:
            print(e)
        conn.commit()
        conn.close()

Connect to given database name and return conn and cursor to CRUD database and/or table.

In [7]:
# Connect to the database
def connect_db(name=None):
    if name == None:
        print("Please enter database name")
    else:
        try:
            conn = pymysql.connect(host=config["localhost"], user=config["user"], password=config["password"], database=name, port=3306)
            cursor = conn.cursor()
            return conn, cursor
        except Exception as e:
            print(e)
        conn.commit()
        conn.close()

With defined function, It is available create and delete database with one line.

In [8]:
create_db(name="studysql")
delete_db(name="studysql")
create_db(name="studysql")

(1007, "Can't create database 'studysql'; database exists")
deleted database studysql
created database studysql


Let's start CRUD table on studysql database

Create table with name USERS can be executed with CREATE TABLE TABLE_NAME operator and it has fields userid, lastname, firstname and city.
userid is primary key and also automatically incremented by 1.

In [24]:
conn, cursor = connect_db(name="studysql")

sql = '''
    CREATE TABLE USERS (
        UserID int NOT NULL AUTO_INCREMENT PRIMARY KEY,
        LastName varchar(255),
        FirstName varchar(255),
        City varchar(255)
    )AUTO_INCREMENT=1;
'''
cursor.execute(sql)
conn.commit()

sql = 'SHOW TABLES'
cursor.execute(sql)
conn.commit()
rows = cursor.fetchall()
print("studysql databse has tables =", rows)

studysql databse has tables = (('users',),)


Read USERS table with SELECT operator, * means all column.
Since there is no row or record, it shows empty table.

In [25]:
sql = 'SELECT * FROM USERS;'
cursor.execute(sql)
conn.commit()
rows = cursor.fetchall()
print("user rows =", rows)

user rows = ()


Update table includes several operators, INSERT INTO, DELETE, WHERE conditional, LIKE s% %s% %s, ORDER BY and more.
Here it covers baisc operators first.

In [26]:
# insert multiple rows into table USERS
sql = '''INSERT INTO USERS (UserID, LastName, FirstName, City) VALUES (0, 'JAME', 'LEBRON', 'LA'), (0, 'JORDAN', 'MICHAEL', 'CHICAGO'), (0, 'BRYANT', 'KOBE', 'LA');'''
cursor.execute(sql)
conn.commit()

# read table with 3 users
sql = 'SELECT * FROM USERS;'
cursor.execute(sql)
rows = cursor.fetchall()
print("user rows =", rows)

print("###################################################")

# delete user with UserID=1
sql = '''DELETE FROM USERS WHERE UserID = 1;'''
cursor.execute(sql)
conn.commit()

# read table with 2 users
sql = 'SELECT * FROM USERS;'
cursor.execute(sql)
rows = cursor.fetchall()
print("user rows =", rows)

print("###################################################")

# read table with last name contain JORDAN
sql = 'SELECT * FROM USERS WHERE LastName LIKE "%JORDAN%";'
cursor.execute(sql)
rows = cursor.fetchall()
print("user rows =", rows)

print("###################################################")

# read table with order by last name
sql = 'SELECT * FROM USERS ORDER BY LastName;'
cursor.execute(sql)
rows = cursor.fetchall()
print("user rows =", rows)

user rows = ((1, 'JAME', 'LEBRON', 'LA'), (2, 'JORDAN', 'MICHAEL', 'CHICAGO'), (3, 'BRYANT', 'KOBE', 'LA'))
###################################################
user rows = ((2, 'JORDAN', 'MICHAEL', 'CHICAGO'), (3, 'BRYANT', 'KOBE', 'LA'))
###################################################
user rows = ((2, 'JORDAN', 'MICHAEL', 'CHICAGO'),)
###################################################
user rows = ((3, 'BRYANT', 'KOBE', 'LA'), (2, 'JORDAN', 'MICHAEL', 'CHICAGO'))


Delete table with DROP operator.

In [23]:
# studysql database has USERS table
sql = 'SHOW TABLES'
cursor.execute(sql)
conn.commit()
rows = cursor.fetchall()
print("studysql databse has tables =", rows)

# delete USERS table
sql = "DROP TABLE USERS"
cursor.execute(sql)
conn.commit()

# studysql database has no table
sql = 'SHOW TABLES'
cursor.execute(sql)
conn.commit()
rows = cursor.fetchall()
print("studysql databse has tables =", rows)


studysql databse has tables = (('users',),)
studysql databse has tables = ()


Let's use sqlalchemy and pandas to convert mysql table to pandas dataframe and vice versa.

In [41]:

import json
import pymysql
import pandas as pd
from sqlalchemy import create_engine

config = open('MySQL_Config.json')
config = json.load(config)


# Pandas Dataframe to MySQL Table
engine = create_engine("mysql+pymysql://"+config['user']+":"+config['password']+"@"+config['localhost']+":3306/studysql?charset=utf8",  encoding='utf-8')

conn = engine.connect()

df = pd.DataFrame(data={'col1': [1, 2], 'col2': [3, 4]})
df.to_sql(name="dataframe_table", con=conn, if_exists='append', index=False)
# if_exists= append/replace/fail
conn.close()

# MySQ Table to Pandas Dataframe
conn, cursor = connect_db(name="studysql")

sql = "SELECT * FROM dataframe_table"
check_table = pd.read_sql(sql, conn)
conn.close()
check_table

,col1,col2
0,1,3
1,2,4
2,1,3
3,2,4
4,1,3
5,2,4
